Description from `kaggle.com`

### MNIST ("Modified National Institute of Standards and Technology") 

Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.

The goal is to correctly identify digits from a dataset of tens of thousands of handwritten images. 

Practice Skills:
- Computer vision fundamentals including simple neural networks
- Classification methods such as SVM and K-nearest neighbors

Acknowledgements: 
More details about the dataset, including algorithms that have been tried on it and their levels of success, can be found at http://yann.lecun.com/exdb/mnist/index.html. The dataset is made available under a Creative Commons Attribution-Share Alike 3.0 license.

### Data

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

### Data loading and preprocessing

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised=True)
# with_info = True - info about dataset
# as_supervised set to True loads the data in 2-tuple structure [input, target]

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# We get the number of validation samples using mnist info (method from tensorflow datasets) as follows:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# This way is only available for tf datasets 
# To get the full number (not a float) we can change it to int:
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# We do the same with the number of test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# We prepare a function to scale images:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

### Modeling

In [4]:
input_size = 784 
output_size = 10 # 10 digits: 0-9
hidden_layer_size = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

### Optimizer and loss function

In [5]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [6]:
NUM_EPOCHS = 8

In [7]:
model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose=2)

Epoch 1/8
540/540 - 6s - loss: 0.2729 - accuracy: 0.9221 - val_loss: 0.1359 - val_accuracy: 0.9622
Epoch 2/8
540/540 - 3s - loss: 0.1073 - accuracy: 0.9674 - val_loss: 0.0931 - val_accuracy: 0.9730
Epoch 3/8
540/540 - 3s - loss: 0.0722 - accuracy: 0.9776 - val_loss: 0.0741 - val_accuracy: 0.9765
Epoch 4/8
540/540 - 3s - loss: 0.0525 - accuracy: 0.9838 - val_loss: 0.0534 - val_accuracy: 0.9845
Epoch 5/8
540/540 - 3s - loss: 0.0417 - accuracy: 0.9868 - val_loss: 0.0460 - val_accuracy: 0.9842
Epoch 6/8
540/540 - 3s - loss: 0.0310 - accuracy: 0.9907 - val_loss: 0.0385 - val_accuracy: 0.9880
Epoch 7/8
540/540 - 3s - loss: 0.0275 - accuracy: 0.9909 - val_loss: 0.0304 - val_accuracy: 0.9898
Epoch 8/8
540/540 - 3s - loss: 0.0225 - accuracy: 0.9926 - val_loss: 0.0244 - val_accuracy: 0.9910


### Testing

In [8]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.0767 - accuracy: 0.9795


In [10]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}'.format(test_loss, test_accuracy*100))

Test loss: 0.08. Test accuracy: 97.95


#### After that step we cannot change anything in the model without reseting everything. Now the model "had seen" all the data including the test data.
#### We got test accuracy very close to the validation accuracy which means we didn't overfit 